In [79]:
#загрузка необходимых библиотек 

import os
import gensim
import numpy as np
import nltk
import scipy
import pandas as pd
import pprint
nltk.download("stopwords")
from nltk.corpus import stopwords
from string import punctuation
from nltk import WordPunctTokenizer
import docx2txt
import gensim.downloader as api
from gensim.models import TfidfModel
from gensim.corpora import Dictionary
from gensim.similarities import MatrixSimilarity
from gensim.models import LsiModel
from gensim.models import Phrases
from nltk.util import ngrams
import rank_bm25

#Определяем токенизатор, выделитель корней слов и стоп-слова для русского языка
tokenizer = WordPunctTokenizer()
stemmer = nltk.stem.SnowballStemmer('russian')
russian_stopwords = stopwords.words("russian")


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Georgy\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
def load_doc(filename):
#загружаем файл в формате docx, разделяем его на отдельные параграфы\абзацы и удаляем пустые абзацы. Выводит массив, содержащий 
#параграфы с формате строк

    doc = docx2txt.process(filename)
    lines = doc.split('\n')
    lines = [line for line in lines if line != '']

    return lines

In [3]:
def build_bigrams(corpus):
#Функция для построения биграмм слов в документах. На выходе: массив из подмассивов, в каждом подмассиве список биграмм из данного параграфа
    corpus_2grams = []
    for doc in corpus:
        doc_2grams = list(ngrams(doc,2))
        doc_2grams = ['_'.join(list(bigram)) for bigram in doc_2grams]
        corpus_2grams.append(doc_2grams)
    return corpus_2grams

In [4]:
def preprocess_corpus(lines, bigrams=False):

#предобрабатываем массив параграфов: токенизируем, удаляем стоп-слова, пустые строки, знаки препинания, удаляем окончания слов.
#на выходе: массив из под-массивов. В одном под-массиве содержатся отдельные слова параграфа в формате строки
#если bigrams=True, функция выдаёт массив подмассивов, где в одном подмассиве находятся биграммы слов одного параграфа
    corp = []
    for parag in lines:
        paragraph = []
        parag = tokenizer.tokenize(parag)
        for word in parag:
            if word not in russian_stopwords and word != ' ' and word.strip() not in punctuation and word.strip()[-1] not in punctuation:
                word = word.lower()
                stemmed_word = stemmer.stem(word)
                paragraph.append(stemmed_word)

        corp.append(paragraph)
    if bigrams:
        corp_bigrams = build_bigrams(corp)
        return corp_bigrams
    else:
        return corp 

In [5]:
def preprocess_query(query, bigrams=False):
#Предобрабатываем запрос по тому же шаблону, что и для целого документа. 
#На вход: пользовательский запрос в формате строки; на выход: Массив из предобработанных слов запроса
#Если bigrams=True, то функция выдаёт биграммы слов в запросе
    query = ' '.join(tokenizer.tokenize(query))
    query_text = []
    for word in query.split():
        if word not in russian_stopwords and word != ' ' and word.strip() not in punctuation and word.strip()[-1] not in punctuation:
            word = word.lower()
            stemmed_word = stemmer.stem(word)
            query_text.append(stemmed_word)
    if bigrams:
        query_bigrams = list(ngrams(query_text,2))
        query_bigrams = ['_'.join(list(bigram)) for bigram in query_bigrams]
        return query_bigrams
    
    else:
        return query_text

In [6]:
def build_tfidf_or_lsi(corpus, method='tfidf'):
    
# построение модели для ранжирования документов. На вход: корпус текстов и метод ("tfidf" или "lsi"). На выход кортеж: (словарь
# терминов в корпусе текстов, оцененная модель и матрица сходств слов) 

    dictionary = Dictionary(corpus)
    corpus_bow = [dictionary.doc2bow(doc) for doc in corpus]
    model_tfidf = TfidfModel(corpus_bow)
    corpus_tfidf = [model_tfidf[doc] for doc in corpus_bow]
    simil_tfidf = MatrixSimilarity(corpus_tfidf)
    if method == 'tfidf':
        
        return dictionary, model_tfidf, simil_tfidf
    
    elif method == 'lsi':
        
        model_lsi = LsiModel(corpus_tfidf,  id2word=dictionary, num_topics=50)
        corpus_lsi = [model_lsi[doc] for doc in corpus_bow]
        simil_lsi = MatrixSimilarity(corpus_lsi)
        
        return dictionary, model_lsi, simil_lsi




In [7]:
def top_docs_tfidf(query, dictionary, model, similarity, ntop=6):

#Находим топ N документов, наиболее близких запросу по критерию cosine similarity. 
#На вход: пользовательский запрос в формате сторки, словарь слов корпуса, модель tfidf и матрица сходств слов (берется из функии build_tfidf_or_lsi)
#На выходе: Отсортированный массив из кортежей с (номер параграфа, значение cosine similarity)
    
    query_corp = dictionary.doc2bow(preprocess_query(query))
    query_tfidf = model[query_corp]
    query_simil = enumerate(similarity[query_tfidf])
    query_top_docs = sorted(query_simil, key=lambda k: -k[1])
    if len(query_top_docs) > ntop: 
        query_top_docs = query_top_docs[:ntop]
    else:
        query_top_docs = query_top_docs
    
    top_docs_indices = [elem[0] for elem in query_top_docs]
    return top_docs_indices 
    

def top_docs_lsi(query, dictionary, model, similarity, ntop=6):
    
#Находим топ N документов, наиболее близких запросу по критерию cosine similarity. 
#На вход: пользовательский запрос в формате сторки, словарь слов корпуса, модель lsi и матрица сходств слов (берется из функии build_tfidf_or_lsi)
#На выходе: Отсортированный массив из кортежей с (номер параграфа, значение cosine similarity)

    query_corp = dictionary.doc2bow(preprocess_query(query))
    query_lsi = model[query_corp]
    query_simil = enumerate(similarity[query_lsi])
    query_top_docs = sorted(query_simil, key=lambda k: -k[1])
    if len(query_top_docs) > ntop: 
        query_top_docs = query_top_docs[:ntop]
    else:
        query_top_docs = query_top_docs
    top_docs_indices = [elem[0] for elem in query_top_docs]
#Вместо этого: нужно выводить номера и названия глав, статей и пунктов, как они указаны в документах - (а также сам текст пунктов?)
    return top_docs_indices 

In [52]:
def top_docs_bm25okapi(query, corp, ntop=6):
    bm25okapi = rank_bm25.BM25Okapi(corp)
    top_docs_indices = np.argsort((bm25okapi.get_scores(preprocess_query(query))))[::-1][:ntop]
    top_docs_indices = list(top_docs_indices)
    return top_docs_indices



## На всякий случай: в rank-bm25 есть ещё BM25L и BM25+. 

In [78]:
doc = load_doc('fz152.docx')
corp = preprocess_corpus(doc)

In [9]:
dictionary, model_tfidf, simil_tfidf = build_tfidf_or_lsi(corp)
dictionary, model_lsi, simil_lsi = build_tfidf_or_lsi(corp, method='lsi')

In [53]:
def display_passages_from_doc(query, corp, doc, method='tfidf'):
    
##Вывод индекса и текста для топ-N пунктов из документа по критерию релевантности запросу. 
##На вход: текстовый запрос в формате строки, документ до предобработки и метод ранжирования пунктов документа
#На выход: индекс и текст пунктов из документа, разделенных пустой строкой
    if method == 'tfidf':
        index_string = top_docs_tfidf(query, dictionary, model_tfidf, simil_tfidf)
        for idx in index_string:
            answer = str(idx) + ': ' + doc[idx]
            pprint.pprint(answer)
            print('\n')
            
    elif method == 'lsi':
        index_string = top_docs_lsi(query, dictionary, model_lsi, simil_lsi)
        for idx in index_string:
            answer = str(idx) + ': ' + doc[idx]
            pprint.pprint(answer)
            print('\n')
            
    elif method == 'bm25okapi':
        index_string = top_docs_bm25okapi(query, corp=corp)
        for idx in index_string:
            answer = str(idx) + ': ' + doc[idx]
            pprint.pprint(answer)
            print('\n')
            
            
    else:
        print('Неправильное значение method. Method может принимать значения \'tfidf\', \'lsi\' или \'bm25okapi\'. ')
        
            

In [53]:
#dictionary.filter_n_most_frequent(5)

In [54]:
display_passages_from_doc('Что делать, если обнаружена неточность или неполнота в персональных данных?', doc, method='bm25okapi')

('27: При обработке персональных данных должны быть обеспечены точность '
 'персональных данных, их достаточность, а в необходимых случаях и '
 'актуальность по отношению к целям обработки персональных данных. Оператор '
 'должен принимать необходимые меры либо обеспечивать их принятие по удалению '
 'или уточнению неполных или неточных данных.')


('176: В случае подтверждения факта неточности персональных данных оператор на '
 'основании сведений, представленных субъектом персональных данных или его '
 'представителем либо уполномоченным органом по защите прав субъектов '
 'персональных данных, или иных необходимых документов обязан уточнить '
 'персональные данные либо обеспечить их уточнение (если обработка '
 'персональных данных осуществляется другим лицом, действующим по поручению '
 'оператора) в течение семи рабочих дней со дня представления таких сведений и '
 'снять блокирование персональных данных.')


('98: Субъект персональных данных имеет право на получение сведений, ука

In [103]:
queries_list = ['Когда субъект может отозвать своё согласие на обработку персональных данных?',
                'Срок ответа обоснование отказа в обработке персональных данных', 
               'Когда запрещена трансграничная передача персональных данных?',
                'Что должно включаться в согласие на обработку персональных данных?', 
                'Кто ответственен за защиту персональных данных?', 
                'Что делать, если обнаружена неточность или неполнота в персональных данных?',
                'Можно ли использовать персональные данные в собственных целях?',
                'Что делать в случае, когда субъект персональных данных недееспособен?',
                'Данные о судимости субъекта персональных данных', 
                'Может ли осуществляться обработка биометрических данных без согласия субъекта?'
                
               ]